In [13]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pyart
import numpy.ma as ma
import matplotlib.ticker as mticker
from matplotlib import colors as c
from matplotlib.colors import ListedColormap,BoundaryNorm
import cartopy.crs as ccrs
import cartopy.feature as cfeat
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import cartopy.geodesic as cargeo
from shapely.geometry import Polygon
import os
import warnings
warnings.filterwarnings('ignore')

In [14]:
import matplotlib.pyplot as plt

filename = 'PHA210208060004.RAWMKZZ'

# create the plot using RadarDisplay (recommended method)
radar = pyart.io.read_sigmet(filename)
display = pyart.graph.RadarDisplay(radar)
fig = plt.figure(figsize=(15,13))
ax1 = fig.add_subplot(221)
display.plot('reflectivity', 0, vmin=8, vmax=64.,cmap=pyart.graph.cm.NWSRef)
ax2 = fig.add_subplot(222)
display.plot('velocity',0,cmap="pyart_NWSVel")
ax3 = fig.add_subplot(223)
display.plot('differential_phase', 0)
ax4 = fig.add_subplot(224)
display.plot('differential_reflectivity',0,cmap="pyart_RefDiff")
#display.plot_range_rings([100, 200, 300])
#display.plot_cross_hair(5.)
plt.show()

In [53]:
radar.fields.keys()

dict_keys(['total_power', 'reflectivity', 'velocity', 'spectrum_width', 'differential_reflectivity', 'specific_differential_phase', 'cross_correlation_ratio', 'normalized_coherent_power', 'differential_phase', 'radar_echo_classification'])

In [21]:
class UF:
    def __init__(self,radar,shape_grid,lat_0, lon_0):
        self.radar = radar
        self.shape = shape_grid
        self.lat_0 = lat_0
        self.lon_0 = lon_0
    def noise_rejection(self):
        mask_noises_CC=ma.masked_less(self.radar.fields['cross_correlation_ratio']['data'],0.85)
        mask_noises_Z=ma.masked_less(self.radar.fields['reflectivity']['data'],0)
        rm_noises_Z_=np.ma.masked_array(self.radar.fields['reflectivity']['data'],mask_noises_CC.mask)
        rm_noises_Z=np.ma.masked_array(rm_noises_Z_,mask_noises_Z.mask)
        self.radar.add_field_like('reflectivity','Z_removed_noises',rm_noises_Z,replace_existing=True)
    def convert_grid(self):
        grid = pyart.map.grid_from_radars(
            self.radar,
            grid_shape=self.shape, #Number of points in the grid (z, y, x)
            grid_limits=((0, 9000), (-200000, 200000), (-200000, 200000)), # min-max tuong duong z,y,x
            grid_origin = (self.lat_0, self.lon_0),
            fields=['Z_removed_noises'],
            roi_func='dist_beam',
            weighting_function='cressman')
        return grid
    def convert_lat_lon(self, grid):
        #conver distance to lat/lon
        for i in range(self.shape[1]):
            geog = pyart.core.cartesian_to_geographic_aeqd(grid.x["data"][i],grid.y["data"][i],self.lon_0, self.lat_0, R=6370997.0)
            grid.x["data"][i] = geog[0]
            grid.y["data"][i] = geog[1]
        return grid
    def plot_MAX_product(self,lons,lats,Zh_max):
        cmap=ListedColormap(['deepskyblue', 'dodgerblue','blue', 'palegreen','lime','limegreen','yellow','orange','darkorange','red','mediumvioletred','darkviolet','black'])
        bounds=[0,10,15,20,25,30,35,40,45,55,60,65,70,80]
        norm = BoundaryNorm(bounds, cmap.N)
        ticks  = [10,15,20,25,30,35,40,45,55,60,65,70]
        circle_points_50km= cargeo.Geodesic().circle(lon=103.51694002747536,lat=21.571390070021152, radius=50000, n_samples=180, endpoint=False)
        circle_points_100km= cargeo.Geodesic().circle(lon=103.51694002747536,lat=21.571390070021152, radius=100000, n_samples=180, endpoint=False)
        circle_points_200km= cargeo.Geodesic().circle(lon=103.51694002747536,lat=21.571390070021152, radius=200000, n_samples=180, endpoint=False)
        geom_50km=Polygon(circle_points_50km)
        geom_100km=Polygon(circle_points_100km)
        geom_200km=Polygon(circle_points_200km)
        #plot map with datas
        fig=plt.figure(figsize=(15,20),dpi=80)
        ax1=fig.add_subplot(1,1,1, projection=ccrs.PlateCarree())
        ax1.add_feature(cfeat.LAND)
        ax1.add_feature(cfeat.OCEAN.with_scale('10m'))
        ax1.add_feature(cfeat.COASTLINE.with_scale('50m'))
        ax1.add_feature(cfeat.BORDERS.with_scale('50m'))
        state_provinces_lines=cfeat.NaturalEarthFeature(category='cultural',name='admin_1_states_provinces_lines',scale='10m',facecolor='none')
        ax1.add_feature(state_provinces_lines,edgecolor='gray')
        g1=ax1.gridlines(draw_labels=True,linestyle='--')
        #g1.xlabels_top = False
        #g1.ylabels_left = False
        g1.xlocator = mticker.FixedLocator([102.51694002747536, 103.51694002747536, 104.51694002747536, 105.51694002747536])
        g1.ylocator = mticker.FixedLocator([20.571390070021152,21.571390070021152,22.571390070021152])
        ax1.set_extent((grid.x['data'][0],grid.x['data'][-1],grid.y['data'][0],grid.y['data'][-1]),crs=ccrs.PlateCarree())
        ax1.add_geometries((geom_50km,), crs=ccrs.PlateCarree(), facecolor='none', edgecolor='black', linewidth=1, linestyle='--')
        ax1.add_geometries((geom_100km,), crs=ccrs.PlateCarree(), facecolor='none', edgecolor='black', linewidth=1, linestyle='--')
        ax1.add_geometries((geom_200km,), crs=ccrs.PlateCarree(), facecolor='none', edgecolor='black', linewidth=1, linestyle='--')
        z1_plot=ax1.pcolormesh(lons,lats,Zh_max,shading='flat',cmap=cmap,norm=norm)#cmap=cmap,norm=norm)
        ax1.scatter(x=103.51694002747536,y=21.571390070021152,transform=ccrs.PlateCarree(), color="black",s=50, alpha=1,marker='P')
        #ax1.scatter(x=105.2796,y=21.16711,transform=ccrs.PlateCarree(), color="red",s=100, alpha=1,marker='o')
        ax1.set(title="Z_MAX_product "+ self.radar.time['units'][14:])
        plt.colorbar(z1_plot,ax=ax1,ticks=ticks,extend='both',label="dBZ",orientation='horizontal')
#Tạo giới hạn bán kính 200km    
y,x = np.ogrid[-200:401-200, -200:401-200]
mask = x*x + y*y > 200*200

In [ ]:
lat_0 = radar.latitude['data'][0]
lon_0 = radar.longitude['data'][0]
shape_grid = (10, 401,401) # grid with z = 10 level (1km), h-v=400 point (1km)
UF_File = UF(radar, shape_grid, lat_0, lon_0)
ref_cor_noise=UF_File.noise_rejection()
grid = UF_File.convert_grid()
grid_lat_lon = UF_File.convert_lat_lon(grid)
lon=grid_lat_lon.x['data']
lat=grid_lat_lon.y['data']
lons,lats=np.meshgrid(lon,lat)
Zh_max=grid.fields['Z_removed_noises']['data']
Zh_max=ma.masked_less_equal(np.amax(Zh_max,axis=0),0.)#np.ma.masked_array(ma.masked_less_equal(np.amax(Zh_max,axis=0),0.),mask)
cmap=ListedColormap(['deepskyblue', 'dodgerblue','blue', 'palegreen','lime','limegreen','yellow','orange','darkorange','red','mediumvioletred','darkviolet','black'])
bounds=[0,10,15,20,25,30,35,40,45,55,60,65,70,80]
norm = BoundaryNorm(bounds, cmap.N)
ticks  = [10,15,20,25,30,35,40,45,55,60,65,70]
circle_points_50km= cargeo.Geodesic().circle(lon=103.51694002747536,lat=21.571390070021152, radius=50000, n_samples=180, endpoint=False)
circle_points_100km= cargeo.Geodesic().circle(lon=103.51694002747536,lat=21.571390070021152, radius=100000, n_samples=180, endpoint=False)
circle_points_200km= cargeo.Geodesic().circle(lon=103.51694002747536,lat=21.571390070021152, radius=200000, n_samples=180, endpoint=False)
geom_50km=Polygon(circle_points_50km)
geom_100km=Polygon(circle_points_100km)
geom_200km=Polygon(circle_points_200km)
#plot map with datas
fig=plt.figure(figsize=(15,20),dpi=80)
ax10=fig.add_subplot(1,1,1, projection=ccrs.PlateCarree())
ax10.add_feature(cfeat.LAND)
ax10.add_feature(cfeat.OCEAN.with_scale('10m'))
ax10.add_feature(cfeat.COASTLINE.with_scale('50m'))
ax10.add_feature(cfeat.BORDERS.with_scale('50m'))
state_provinces_lines=cfeat.NaturalEarthFeature(category='cultural',name='admin_1_states_provinces_lines',scale='10m',facecolor='none')
ax10.add_feature(state_provinces_lines,edgecolor='gray')
g1=ax10.gridlines(draw_labels=True,linestyle='--')
g1.xlocator = mticker.FixedLocator([102.51694002747536, 103.51694002747536, 104.51694002747536, 105.51694002747536])
g1.ylocator = mticker.FixedLocator([20.571390070021152,21.571390070021152,22.571390070021152])
ax10.set_extent((grid.x['data'][0],grid.x['data'][-1],grid.y['data'][0],grid.y['data'][-1]),crs=ccrs.PlateCarree())
ax10.add_geometries((geom_50km,), crs=ccrs.PlateCarree(), facecolor='none', edgecolor='black', linewidth=1, linestyle='--')
ax10.add_geometries((geom_100km,), crs=ccrs.PlateCarree(), facecolor='none', edgecolor='black', linewidth=1, linestyle='--')
ax10.add_geometries((geom_200km,), crs=ccrs.PlateCarree(), facecolor='none', edgecolor='black', linewidth=1, linestyle='--')
z1_plot=ax10.pcolormesh(lons,lats,Zh_max,shading='flat',cmap=cmap,norm=norm)#cmap=cmap,norm=norm)
#ax1.scatter(x=103.51694002747536,y=21.571390070021152,transform=ccrs.PlateCarree(), color="black",s=50, alpha=1,marker='P')
#ax1.set(title="Z_MAX_product "+ radar.time['units'][14:])
#plt.colorbar(z1_plot,ax=ax1,ticks=ticks,extend='both',label="dBZ",orientation='horizontal')